In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 读取CSV文件
df1 = pd.read_csv('训练集.csv')
df2 = pd.read_csv('测试集.csv')
# 数据预处理
scaler = StandardScaler()
df1[['longitude', 'latitude']] = scaler.fit_transform(df1[['longitude', 'latitude']])
df2[['longitude', 'latitude']] = scaler.fit_transform(df2[['longitude', 'latitude']])
df1[['value']] = scaler.fit_transform(df1[['value']])
df2[['value']] = scaler.fit_transform(df2[['value']])
inputs1 = df1[['longitude', 'latitude']].values
outputs1 = df1['value'].values
inputs2 = df2[['longitude', 'latitude']].values
outputs2 = df2['value'].values

# 划分训练集和测试集
# X_train, X_test, y_train, y_test = train_test_split(inputs1,inputs2,outputs1, outputs2,  random_state=42)
X_train = inputs1
X_test = inputs2
y_train = outputs1
y_test = outputs2
# 转换为PyTorch张量
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # 保持二维形状
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)  # 保持二维形状
# print(X_train,y_train,X_test,y_test)

In [2]:

# 定义RNN模型
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=3):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h0):

        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# 初始化模型和损失函数、优化器
input_size = 2  # 经度和纬度
output_size = 1  # 单值输出
hidden_size = 128
num_layers = 3
model = RNNModel(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
# criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [3]:
# class RNNModel(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size, num_layers=4):
#         super(RNNModel, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_size, output_size)
# 
#     def forward(self, x, h0):
#         # x: (batch_size, sequence_length, input_size)
#         # h0: (num_layers, batch_size, hidden_size)
#         out, _ = self.rnn(x, h0)
#         # out: (batch_size, sequence_length, hidden_size)
#         out = self.fc(out[:, -1, :])  # 只取序列的最后一个输出作为模型输出
#         # out: (batch_size, output_size)
#         return out
# 
# # 假设 batch_size 为 32，sequence_length 为 10
# batch_size = 32
# sequence_length = 10
# input_size = 2  # 经度和纬度
# output_size = 1  # 单值输出
# hidden_size = 256
# num_layers =4
# 
# # 初始化模型和损失函数、优化器
# model = RNNModel(input_size, hidden_size, output_size)
# # criterion = nn.LiLoss()
# criterion = nn.L1Loss()
# # criterion = nn.SmoothL1Loss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# 
# # 随机生成一个 batch 的输入数据和初始隐藏状态
# input_data = torch.rand(batch_size, sequence_length, input_size)
# h0 = torch.zeros(num_layers, batch_size, hidden_size)
# 
# # 计算模型输出和损失
# output = model(input_data, h0)
# target = torch.rand(batch_size, output_size)  # 随机生成一个 batch 的目标值
# loss = criterion(output, target)
# 
# # 进行反向传播和参数更新
# optimizer.zero_grad()
# loss.backward()
# optimizer.step()


In [0]:

# 训练模型
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs_pred = model(X_train.unsqueeze(1), torch.zeros(num_layers, X_train.size(0), hidden_size))
    loss = criterion(outputs_pred, y_train)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
# 模型预测

In [9]:

with torch.no_grad():
    y_pred = model(X_test.unsqueeze(1), torch.zeros(num_layers, X_test.size(0), hidden_size))
  
      # 将预测值反标准化
    y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))

    y_pred = torch.tensor(y_pred)
    diff = y_pred - y_test
    std_diff = torch.std(diff)
      # df = pd.DataFrame(y_pred, columns=['prediction'])
    # df.to_csv('predictions.csv', index=False)

    # 将二维数组转换为一维数组
    predictions_1d = y_pred.reshape(-1)
    # 创建包含预测结果的 DataFrame
    import numpy as np



    # 保留小数点后两位
    y_pred_RNN = np.round(predictions_1d, 2)
    df = pd.DataFrame({'prediction': y_pred_RNN},index=range(len(y_pred_RNN)))
    desktop_path = r'D:\桌面内容\深度学习的资料\GIthub代码\LSTM-SVM-RF-time-series-master\LSTM-SVM-RF-time-series-master\重力插值-数据2\prediction1-RNN-1-1-1.csv'
    #将 DataFrame 写入 CSV 文件
    df.to_csv(desktop_path, index=False)

    print('差值的标准差:', std_diff.item())
    # print(y_pred)
    # print("Predictions:", y_pred.view(-1).tolist())
    # print("Actual:", y_test.view(-1).tolist())
with torch.no_grad():
    y_pred = model(X_test.unsqueeze(1), torch.zeros(num_layers, X_test.size(0), hidden_size))
    y_pred_tensor = torch.tensor(y_pred)
    diff = y_pred_tensor - torch.Tensor(y_test)
    # diff = y_pred - y_test
    diff_mean = torch.mean(diff)
    # print(y_pred)

    # 假设 predictions 是一个包含预测结果的列表或数组
    df = pd.DataFrame(y_pred, columns=['prediction'])
    df.to_csv('predictions.csv', index=False)


# 如果有对应的索引，可以将索引加入 DataFrame
# df.index = your_index

    print('差值的均值:', diff_mean.item())
    # print(y_pred)


差值的标准差: 28.19388115603303
差值的均值: -0.002336864359676838


C:\Users\94430\AppData\Local\Temp\ipykernel_64696\2096155254.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred_tensor = torch.tensor(y_pred)


In [5]:
# with torch.no_grad():
#     y_pred = model(X_test.unsqueeze(1), torch.zeros(num_layers, X_test.size(0), hidden_size))# 假设 y_pred 是模型的预测结果
#     # 将 y_pred 转换为一维数组
#     y_pred = y_pred.view(-1)
# 
#     y_pred_unscaled = scaler.inverse_transform(y_pred)
#     y_test_unscaled = scaler.inverse_transform(y_test)
#     diff = y_pred_unscaled - y_test_unscaled
#     diff_mean = torch.mean(diff)
#     print(y_pred)
#     print('差值的均值:', diff_mean.item())
# # 打印反标准化后的数据
# print("Predictions (unscaled):", y_pred_unscaled)


In [6]:
# with torch.no_grad():
#     y_pred = model(X_test.unsqueeze(1), torch.zeros(num_layers, X_test.size(0), hidden_size))
#     y_pred_tensor = torch.tensor(y_pred)
#     diff = y_pred_tensor - torch.Tensor(y_test)
#     # diff = y_pred - y_test
#     diff_mean = torch.mean(diff)
#     # print(y_pred)
# 
#     # 假设 predictions 是一个包含预测结果的列表或数组
#     df = pd.DataFrame(y_pred, columns=['prediction'])
#     df.to_csv('predictions.csv', index=False)
# 
# 
# # 如果有对应的索引，可以将索引加入 DataFrame
# # df.index = your_index
# 
#     print('差值的均值:', diff_mean.item())
#     # print(y_pred)


In [7]:
# import os
# print(os.getcwd())



D:\桌面内容\深度学习的资料\GIthub代码\LSTM-SVM-RF-time-series-master\LSTM-SVM-RF-time-series-master\重力插值-数据2


In [8]:

# # 模型预测
# with torch.no_grad():
#     y_pred = model(X_test.unsqueeze(1), torch.zeros(num_layers, X_test.size(0), hidden_size))
#   
#     diff = y_pred - y_test
#     std_diff = torch.std(diff)
#       # df = pd.DataFrame(y_pred, columns=['prediction'])
#     # df.to_csv('predictions.csv', index=False)
# 
#     # 将二维数组转换为一维数组
#     predictions_1d = y_pred.reshape(-1)
#     # 创建包含预测结果的 DataFrame
#     df = pd.DataFrame({'prediction': predictions_1d},index=range(len(predictions_1d)))
#     desktop_path = r'D:\桌面内容\深度学习的资料\GIthub代码\LSTM-SVM-RF-time-series-master\LSTM-SVM-RF-time-series-master\重力插值-数据2\predictions-RNN-13(huber).csv'
#     # 将 DataFrame 写入 CSV 文件
#     df.to_csv(desktop_path, index=False)
# 
#     # print('差值的标准差:', std_diff.item())
#     # print(y_pred)
#     # # print("Predictions:", y_pred.view(-1).tolist())
#     # # print("Actual:", y_test.view(-1).tolist())
